In [2]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
Z = np.ones(4*1000000, np.float32)

In [4]:
Z.size, Z.shape

(4000000, (4000000,))

### Memory Layout

In [7]:
Z = np.arange(9).reshape(3, 3).astype(np.int16)
Z

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]], dtype=int16)

In [8]:
Z.size, Z.shape

(9, (3, 3))

In [10]:
Z.itemsize

2

In [11]:
Z.ndim

2

In [12]:
len(Z.shape) # ~ Z.ndim

2

In [13]:
# Z is not a view

In [17]:
strides = Z.shape[1] * Z.itemsize, Z.itemsize
strides

(6, 2)

In [18]:
Z.strides

(6, 2)

### Item Layout

```python
               shape[1]
                 (=3)
            ┌───────────┐

         ┌  ┌───┬───┬───┐  ┐
         │  │ 0 │ 1 │ 2 │  │
         │  ├───┼───┼───┤  │
shape[0] │  │ 3 │ 4 │ 5 │  │ len(Z)
 (=3)    │  ├───┼───┼───┤  │  (=3)
         │  │ 6 │ 7 │ 8 │  │
         └  └───┴───┴───┘  ┘
```

### Flattened Item Layout

```python
┌───┬───┬───┬───┬───┬───┬───┬───┬───┐
│ 0 │ 1 │ 2 │ 3 │ 4 │ 5 │ 6 │ 7 │ 8 │
└───┴───┴───┴───┴───┴───┴───┴───┴───┘

└───────────────────────────────────┘
               Z.size
                (=9)
```

### Memory Layout

```python
                         strides[1]
                           (=2)
                  ┌─────────────────────┐

          ┌       ┌──────────┬──────────┐ ┐
          │ p+00: │ 00000000 │ 00000000 │ │
          │       ├──────────┼──────────┤ │
          │ p+02: │ 00000000 │ 00000001 │ │ strides[0]
          │       ├──────────┼──────────┤ │   (=2x3)
          │ p+04  │ 00000000 │ 00000010 │ │
          │       ├──────────┼──────────┤ ┘
          │ p+06  │ 00000000 │ 00000011 │
          │       ├──────────┼──────────┤
Z.nbytes  │ p+08: │ 00000000 │ 00000100 │
(=3x3x2)  │       ├──────────┼──────────┤
          │ p+10: │ 00000000 │ 00000101 │
          │       ├──────────┼──────────┤
          │ p+12: │ 00000000 │ 00000110 │
          │       ├──────────┼──────────┤
          │ p+14: │ 00000000 │ 00000111 │
          │       ├──────────┼──────────┤
          │ p+16: │ 00000000 │ 00001000 │
          └       └──────────┴──────────┘

                  └─────────────────────┘
                        Z.itemsize
                     Z.dtype.itemsize
                           (=2)
```

If we now take a slice of Z, the result is a view of the base array Z

In [20]:
V = Z[::2, ::2]
V

array([[0, 2],
       [6, 8]], dtype=int16)

In [21]:
Z

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]], dtype=int16)

In [22]:
V[0, 0] = 100

In [23]:
V

array([[100,   2],
       [  6,   8]], dtype=int16)

In [24]:
Z

array([[100,   1,   2],
       [  3,   4,   5],
       [  6,   7,   8]], dtype=int16)

In [26]:
V[1, 1] = 200
V

array([[100,   2],
       [  6, 200]], dtype=int16)

In [27]:
Z

array([[100,   1,   2],
       [  3,   4,   5],
       [  6,   7, 200]], dtype=int16)

### Item Layout

```python
               shape[1]
                 (=2)
            ┌───────────┐

         ┌  ┌───┬╌╌╌┬───┐  ┐
         │  │ 0 │   │ 2 │  │            ┌───┬───┐
         │  ├───┼╌╌╌┼───┤  │            │ 0 │ 2 │
shape[0] │  ╎   ╎   ╎   ╎  │ len(Z)  →  ├───┼───┤
 (=2)    │  ├───┼╌╌╌┼───┤  │  (=2)      │ 6 │ 8 │
         │  │ 6 │   │ 8 │  │            └───┴───┘
         └  └───┴╌╌╌┴───┘  ┘
```

### Flattened Item Layout

```python
┌───┬╌╌╌┬───┬╌╌╌┬╌╌╌┬╌╌╌┬───┬╌╌╌┬───┐       ┌───┬───┬───┬───┐
│ 0 │   │ 2 │   ╎   ╎   │ 6 │   │ 8 │   →   │ 0 │ 2 │ 6 │ 8 │
└───┴╌╌╌┴───┴╌╌╌┴╌╌╌┴╌╌╌┴───┴╌╌╌┴───┘       └───┴───┴───┴───┘
└─┬─┘   └─┬─┘           └─┬─┘   └─┬─┘
  └───┬───┘               └───┬───┘
      └───────────┬───────────┘
               Z.size
                (=4)
```

### Memory Layout

```python
              ┌        ┌──────────┬──────────┐ ┐             ┐
            ┌─┤  p+00: │ 00000000 │ 00000000 │ │             │
            │ └        ├──────────┼──────────┤ │ strides[1]  │
          ┌─┤    p+02: │          │          │ │   (=4)      │
          │ │ ┌        ├──────────┼──────────┤ ┘             │
          │ └─┤  p+04  │ 00000000 │ 00000010 │               │
          │   └        ├──────────┼──────────┤               │ strides[0]
          │      p+06: │          │          │               │   (=12)
          │            ├──────────┼──────────┤               │
Z.nbytes ─┤      p+08: │          │          │               │
  (=8)    │            ├──────────┼──────────┤               │
          │      p+10: │          │          │               │
          │   ┌        ├──────────┼──────────┤               ┘
          │ ┌─┤  p+12: │ 00000000 │ 00000110 │
          │ │ └        ├──────────┼──────────┤
          └─┤    p+14: │          │          │
            │ ┌        ├──────────┼──────────┤
            └─┤  p+16: │ 00000000 │ 00001000 │
              └        └──────────┴──────────┘

                       └─────────────────────┘
                             Z.itemsize
                          Z.dtype.itemsize
                                (=2)
```

# Views and copys

```
Views and copies are important concepts for the optimization of your numerical computations, the whole story is a bit more complex.
```

### Direct and Indirect Access

```
- Indexing and fancy indexing.

- The first will always return a view, while the second willl return a copy.

- This different is important because in the first case, modifying the view will modifying the base array while this is not true in the second case.
```

In [28]:
Z = np.zeros(9)
Z

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [30]:
Z_view = Z[:3]
Z_view

array([ 0.,  0.,  0.])

In [31]:
Z_view[...] = 1
Z_view

array([ 1.,  1.,  1.])

In [32]:
Z

array([ 1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.])

In [34]:
Z = np.zeros(9)
Z

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [35]:
Z_copy = Z[[0, 1, 2]]
Z_copy

array([ 0.,  0.,  0.])

In [37]:
Z_copy[...] = 1
Z_copy

array([ 1.,  1.,  1.])

In [38]:
Z

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [39]:
Z = np.zeros(9)
Z

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [40]:
index = [0, 1, 2]

In [41]:
Z[index] = 1
Z

array([ 1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.])

```
How to check if a result of indexing is a view or a copy
```

In [43]:
Z = np.random.uniform(0, 1, (5,5))
Z

array([[ 0.75746164,  0.85716096,  0.7128271 ,  0.80552063,  0.15890118],
       [ 0.85684153,  0.18946721,  0.84141351,  0.35398471,  0.1718219 ],
       [ 0.27998666,  0.92566015,  0.68705064,  0.82269625,  0.52059879],
       [ 0.40115661,  0.15548791,  0.624396  ,  0.44531708,  0.6419519 ],
       [ 0.59329404,  0.30753693,  0.75223688,  0.48642377,  0.13990487]])

In [44]:
Z1 = Z[:3, :]
Z1

array([[ 0.75746164,  0.85716096,  0.7128271 ,  0.80552063,  0.15890118],
       [ 0.85684153,  0.18946721,  0.84141351,  0.35398471,  0.1718219 ],
       [ 0.27998666,  0.92566015,  0.68705064,  0.82269625,  0.52059879]])

In [45]:
Z2 = Z[[0, 1, 2], :]
Z2

array([[ 0.75746164,  0.85716096,  0.7128271 ,  0.80552063,  0.15890118],
       [ 0.85684153,  0.18946721,  0.84141351,  0.35398471,  0.1718219 ],
       [ 0.27998666,  0.92566015,  0.68705064,  0.82269625,  0.52059879]])

In [46]:
np.allclose?

In [47]:
# Returns True if two arrays are element-wise equal within a tolerance.
np.allclose(Z1, Z2)

True

In [49]:
Z1.base is Z

True

In [50]:
Z2.base is Z

False

In [51]:
Z2.base is None

True

```
__NOTE__: if it is None, then result is a __copy__
```

```
Some function in numpy return a view (ravel), while some others always return a copy(flatten)
```

In [53]:
Z = np.zeros((5, 5))
Z

array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]])

In [56]:
# Return a flattened array.
Z.ravel?

In [57]:
np.ravel?

In [58]:
Z.ravel().base is Z

True

In [59]:
Z[::2, ::2].ravel().base is Z

False

### Temporary Copy

In [60]:
X = np.ones(10, dtype=np.int)
Y = np.ones(10, dtype=np.int)

In [61]:
X

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [63]:
Y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [65]:
A = 2 * X + 2 * Y
A

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

```
In the example above, three intermediate arrays have been created. One for holding the result of 2*X, one for holding the result of 2*Y and the last one for holding the result of 2*X+2*Y. In this specific case, the arrays are small enough and this does not really make a difference. However, if your arrays are big, then you have to be careful with such expressions and wonder if you can do it differently. For example, if only the final result matters and you don't need X nor Y afterwards, an alternate solution would be:
```

In [69]:
X = np.ones(10, dtype=int)
Y = np.ones(10, dtype=int)

In [70]:
np.multiply(X, 2, out=X)

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [71]:
np.multiply(Y, 2, out=Y)

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [72]:
np.add(X, Y, out=X)

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

```
Using this alternate solution, no temporary array has been created
```